### **title 기반 Smart Genre Relevance Score (지능형 장르 적합도) 설계**

오디오속성적합도 -> 스포티파이 권한 이슈로 불가 -> 장르적합도 -> 스포티파이가 각 노래 별 장르 제공 안해줌, 아티스트 별 장르만 제공함 -> 아티스트 장르 기반 적합도 -> GPT의 타켓장르(키워드)랑 실제 아티스트 장르와의 매치가 딱 들어맞기 어려워 fail이 너무 많이 발생함+ 일일이 룰을 짜는 건 한계가 있음 -> 데이터소스(스포티파이 아티스트 장르)는 객관적 사실을 쓰되, 판단은 AI를 쓰는 하이브리드 방식 -> **LLM 기반 장르 적합도 평가**(유지보수가 필요 없음)

- **Goal :** 접근 가능한 '아티스트 장르(Artist Genres)' 데이터(Fact)와 LLM의 추론 능력을 결합하여, GPT 추천 결과가 상황에 논리적으로 부합하는지 검증하는 대리 지표(Proxy Metric) 도입. 평가의 핵심 근거가 LLM의 상상이 아닌 **Spotify 데이터베이스(Fact)**에서 오므로 객관적 근거를 가짐.

- **Method**

접근 방식: 외부 데이터 기반 검증 (External Data Verification)

1. GPT 의도 파악 (Intent): GPT가 추천 시 설정한 `Primary Tag`(예: focus_instrumental)와 `Reasoning`을 추출.
2. Ground Truth 수집 (Fact): Spotify API를 통해 해당 아티스트의 공식 `Genres` 리스트(예: ['soundtrack', 'piano'])를 조회.
3. LLM 판단 (Judgment): 수집된 **실제 장르(Fact)**가 의도한 분위기(Intent) 범주에 논리적으로 포함되는지 LLM(Judge)이 판정.
    - *질문 예시: "실제 장르인 'Death Metal'이 의도한 'Sleep' 분위기에 적합한가?" → FAIL*

- **Target** 

목표치: 90% 이상 (Pass Rate)

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from openai import OpenAI # LLM 사용
import json
import time

# ===========================================================
# 1. 설정
# ===========================================================
SPOTIFY_CLIENT_ID = "."
SPOTIFY_CLIENT_SECRET = "."
OPENAI_API_KEY = "--" # 여기에 OpenAI API Key 입력 (GPT에게 물어봐야 하니까!)

INPUT_FILE = "search_hit_rate_result.xlsx"
OUTPUT_FILE = "genre_relevance_final.xlsx"

# ===========================================================
# 2. 연결 (Spotify & OpenAI)
# ===========================================================
try:
    auth_manager = SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET
    )
    sp = spotipy.Spotify(auth_manager=auth_manager)
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API 연결 성공! (Smart Genre Check Mode)")
except Exception as e:
    print(f"❌ 연결 실패: {e}")
    exit()

# ===========================================================
# 3. [핵심] LLM에게 장르 적합성 판단 맡기기
# ===========================================================
def check_genre_fit_with_llm(artist_name, actual_genres, intent_tag, scenario):
    """
    GPT-4o-mini (싸고 빠름)를 사용해서 
    Spotify 장르 리스트가 의도(Intent)와 맞는지 판단함.
    """
    # 장르 정보가 없으면 판단 불가
    if not actual_genres:
        return "UNKNOWN", "No genre data on Spotify"

    prompt = f"""
    You are a Music Genre Expert.
    
    # Context
    - Scenario: {scenario}
    - Intended Vibe (Tag): "{intent_tag}"
    - Artist: "{artist_name}"
    - Actual Spotify Genres: {actual_genres}

    # Task
    Determine if the 'Actual Spotify Genres' are compatible with the 'Intended Vibe'.
    (e.g., If Intent='Relax' and Genres=['jazz', 'ballad'], then PASS. If Genres=['death metal'], then FAIL.)

    # Output Format (JSON)
    {{"result": "PASS" or "FAIL", "reason": "short reason"}}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", # 가벼운 모델 사용
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            temperature=0.0
        )
        res_json = json.loads(response.choices[0].message.content)
        return res_json['result'], res_json['reason']
    except Exception as e:
        return "ERROR", str(e)

# ===========================================================
# 4. 메인 실행
# ===========================================================
print(f"\n🚀 Smart Genre Relevance 분석 시작... (파일: {INPUT_FILE})")

try:
    df = pd.read_excel(INPUT_FILE)
except:
    print("❌ 파일을 찾을 수 없습니다.")
    exit()

results = []
pass_count = 0
analyzed_count = 0

for index, row in df.iterrows():
    track_id = row.get('Spotify ID')
    search_status = row.get('Spotify Search')
    
    if search_status != 'SUCCESS' or pd.isna(track_id):
        results.append({**row, "Genre Status": "SKIPPED"})
        continue
        
    analyzed_count += 1
    
    try:
        # 1. Spotify에서 장르 가져오기 (Fact)
        track_info = sp.track(track_id)
        artist_id = track_info['artists'][0]['id']
        artist_info = sp.artist(artist_id)
        actual_genres = artist_info.get('genres', []) 
        
        # 2. GPT 의도 가져오기 (Intent)
        primary_tag = row.get('Primary Tag', 'Unknown')
        scenario = row.get('Scenario', '')
        
        print(f"🔍 Analyzing: {row['Real Artist']} (Tag: {primary_tag}) ...", end=" ")

        # 3. LLM에게 판단 요청 (Judgment) -> 수동 매핑 필요 없음!
        status, reason = check_genre_fit_with_llm(row['Real Artist'], actual_genres, primary_tag, scenario)
        
        if status == "PASS":
            pass_count += 1
            print("✅ PASS")
        else:
            print(f"❌ FAIL ({reason})")

        results.append({
            **row,
            "Actual Genres": str(actual_genres),
            "Genre Status": status,
            "Genre Reason": reason
        })

    except Exception as e:
        print(f"⚠️ Error: {e}")
        results.append({**row, "Genre Status": "ERROR"})
        
    time.sleep(0.3) 

# 결과 저장
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

score = (pass_count / analyzed_count) * 100 if analyzed_count > 0 else 0

print("\n" + "="*50)
print(f"✅ 분석 완료! 파일: {OUTPUT_FILE}")
print(f"🎸 [Smart KPI] Genre Relevance Score: {score:.2f}%")
print("="*50)

✅ API 연결 성공! (Smart Genre Check Mode)

🚀 Smart Genre Relevance 분석 시작... (파일: search_hit_rate_result.xlsx)
🔍 Analyzing: Ludovico Einaudi (Tag: focus) ... ✅ PASS
🔍 Analyzing: Kanye West (Tag: energetic) ... ✅ PASS
🔍 Analyzing: Max Richter (Tag: calm) ... ✅ PASS
🔍 Analyzing: Bill Evans (Tag: relax) ... ✅ PASS
🔍 Analyzing: NewJeans (Tag: energetic) ... ✅ PASS
🔍 Analyzing: Joakim Karud (Tag: focus_lofi) ... ✅ PASS
🔍 Analyzing: Dua Lipa (Tag: energetic) ... ✅ PASS
🔍 Analyzing: IU (Tag: consolation) ... ✅ PASS
🔍 Analyzing: Animals As Leaders (Tag: instrumental metal) ... ✅ PASS
🔍 Analyzing: Ludwig van Beethoven (Tag: energetic) ... ❌ FAIL (The genres 'classical', 'classical piano', and 'chamber music' are generally not energetic and are more suited for relaxation or contemplation.)
🔍 Analyzing: J. Cole (Tag: calm) ... ❌ FAIL (Rap is generally more energetic and not typically associated with a calm vibe.)
🔍 Analyzing: BTS (Tag: focus) ... ❌ FAIL (K-pop is generally upbeat and energetic, which 